# Cassandra

Dataset : taxi_trip_data.csv

## Connection

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': './cassandra_needs/secure-connect-taxi-trip.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("./cassandra_needs/taxi_trip-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.0.6816


## Import and Read

In [3]:
import pandas as pd

taxi_trip_data = pd.read_csv('./datasets/taxi_trip_data.csv', nrows=50_000)

taxi_zone_geo_data = pd.read_csv('./datasets/taxi_zone_geo.csv')

In [4]:
taxi_trip_data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'rate_code', 'store_and_fwd_flag', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'imp_surcharge', 'total_amount', 'pickup_location_id',
       'dropoff_location_id'],
      dtype='object')

In [5]:
taxi_zone_geo_data.columns

Index(['zone_id', 'zone_name', 'borough', 'zone_geom'], dtype='object')

## Data Preprocessing

### Drop Unnecessary Columns

In [6]:
taxi_trip_data.drop(
    columns=["store_and_fwd_flag", "rate_code", "total_amount"], inplace=True
)

### Drop rows with missing essential details


In [7]:
essential_columns = [
    "vendor_id",
    "pickup_datetime",
    "dropoff_datetime",
    "passenger_count",
    "trip_distance",
    "payment_type",
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "imp_surcharge",
    "pickup_location_id",
    "dropoff_location_id",
]


### Drop Missing Values

In [8]:
taxi_trip_data.dropna(subset=essential_columns, inplace=True)
taxi_zone_geo_data.dropna(inplace=True)

## Remove IDs in pickup location IDs that are not present in zone IDs

In [10]:
taxi_trip_data = taxi_trip_data[taxi_trip_data['pickup_location_id'].isin(taxi_zone_geo_data['zone_id'])]

## Join Datasets

In [11]:
joined_data = pd.merge(
    taxi_trip_data,
    taxi_zone_geo_data,
    how="inner",
    left_on="pickup_location_id",
    right_on="zone_id",
)

In [12]:
joined_data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'imp_surcharge', 'pickup_location_id',
       'dropoff_location_id', 'zone_id', 'zone_name', 'borough', 'zone_geom'],
      dtype='object')

## Cassandra

### Initiation

In [13]:
session.set_keyspace("taxi_trip_data")
row = cluster.metadata.keyspaces["taxi_trip_data"]

from pprint import pprint

pprint(row)

### Table Creation

In [14]:
session.execute(
    """
    CREATE TABLE IF NOT EXISTS taxi_trip_data.trip_data (
        trip_id int PRIMARY KEY,
        zone_name text,
        pickup_datetime TIMESTAMP,
        dropoff_datetime TIMESTAMP,
        passenger_count INT,
        payment_type TEXT,
        fare_amount DOUBLE,
        extra DOUBLE,
        mta_tax DOUBLE,
        tip_amount DOUBLE,
        tolls_amount DOUBLE,
        imp_surcharge DOUBLE,
    )
    """
)

In [16]:
joined_data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'imp_surcharge', 'pickup_location_id',
       'dropoff_location_id', 'zone_id', 'zone_name', 'borough', 'zone_geom'],
      dtype='object')

### Insertion

In [17]:
import csv

ROW_LIMIT = 20_000
row_count = 0

with open("./datasets/joined_taxi_trip_data.csv", "r") as f:
    reader = csv.reader(f)  # Create a reader object.
    next(reader)  # Skip the header row.
    for row in reader:
        if row_count >= ROW_LIMIT:
            break
        trip_id = row_count
        zone_name = row[15]
        pickup_datetime = row[1]
        dropoff_datetime = row[2]
        passenger_count = row[3]
        payment_type = row[6]
        fare_amount = row[7]
        extra = row[8]
        mta_tax = row[9]
        tip_amount = row[10]
        tolls_amount = row[11]
        imp_surcharge = row[12]
        pickup_location_id = row[13]

        
        
        query = (
            f"INSERT INTO taxi_trip_data.trip_data (trip_id, zone_name, pickup_datetime, dropoff_datetime, passenger_count, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge) VALUES ({trip_id}, '{zone_name}', '{pickup_datetime}', '{dropoff_datetime}', {passenger_count}, '{payment_type}', {fare_amount}, {extra}, {mta_tax}, {tip_amount}, {tolls_amount}, {imp_surcharge})"
        )
        try:
            session.execute(query)
        except Exception as e:
            print(e)
            break
        row_count += 1

KeyboardInterrupt: 

## Calculations

### Trip Duration

In [11]:
# calculate trip duration

trip_duration_query = """
SELECT trip_id, pickup_datetime, dropoff_datetime 
FROM taxi_trip_data.trip_data
"""

trip_duration = session.execute(trip_duration_query)

trip_duration_df = pd.DataFrame(list(trip_duration))

trip_duration_df["trip_duration"] = trip_duration_df["dropoff_datetime"] - trip_duration_df["pickup_datetime"]

trip_duration_df["trip_duration"] = trip_duration_df["trip_duration"].dt.total_seconds()

trip_duration_df["trip_duration"] = trip_duration_df["trip_duration"].astype(int)

# update trip duration

for index, row in trip_duration_df.iterrows():
    trip_id = row["trip_id"]
    trip_duration = row["trip_duration"]
    query = f"UPDATE taxi_trip_data.trip_data SET trip_duration = {trip_duration} WHERE trip_id = {trip_id}"
    session.execute(query)

### Total Trip Cost

In [12]:
# calculate total trip cost

total_trip_cost_query = """
SELECT trip_id, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge
FROM taxi_trip_data.trip_data
"""

total_trip_cost = session.execute(total_trip_cost_query)

total_trip_cost_df = pd.DataFrame(list(total_trip_cost))

total_trip_cost_df["total_trip_cost"] = total_trip_cost_df[["fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "imp_surcharge"]].sum(axis=1)

# create total trip cost column in the table

session.execute(
    """
    ALTER TABLE taxi_trip_data.trip_data ADD total_trip_cost DOUBLE
    """
)

# update total trip cost

for index, row in total_trip_cost_df.iterrows():
    trip_id = row["trip_id"]
    total_trip_cost = row["total_trip_cost"]
    query = f"UPDATE taxi_trip_data.trip_data SET total_trip_cost = {total_trip_cost} WHERE trip_id = {trip_id}"
    session.execute(query)

## What is the most common payment type used per time of day?
Hint: time of day meaning morning, afternoon or evening

In [ ]:
## What is the most common payment type used per time of day?
# Hint: time of day meaning morning, afternoon or evening

payment_type_query = """
SELECT
    pickup_datetime,
    payment_type
FROM
    taxi_trip_data.trip_data
"""

payment_type = session.execute(payment_type_query)

payment_type_df = pd.DataFrame(list(payment_type))

payment_type_df["pickup_datetime"] = pd.to_datetime(payment_type_df["pickup_datetime"])

payment_type_df["hour"] = payment_type_df["pickup_datetime"].dt.hour

payment_type_df["time_of_day"] = pd.cut(
    payment_type_df["hour"],
    bins=[0, 6, 12, 18, 24],
    labels=["night", "morning", "afternoon", "evening"],
    right=False,
)

In [ ]:
payment_type_df.drop(columns=["pickup_datetime", "hour"], inplace=True)

payment_type_df = (
    payment_type_df.groupby(["time_of_day", "payment_type"])
    .size()
    .reset_index(name="count")
)

payment_type_df = payment_type_df.sort_values(
    by=["time_of_day", "count"], ascending=[True, False]
)

payment_type_df = payment_type_df.groupby("time_of_day").head(1)

payment_type_df

### Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

for time_of_day, data in payment_type_df.groupby("time_of_day"):
    ax.bar(data["payment_type"], data["count"], label=time_of_day)

ax.set_xlabel("Payment Type")
ax.set_ylabel("Count")
ax.set_title("Most Common Payment Type Used Per Time of Day")
ax.legend()
plt.show()

## What is the average tip amount per passenger count? 

In [ ]:
## What is the average tip amount per passenger count?

tip_amount_query = """
SELECT
    passenger_count,
    tip_amount
FROM
    taxi_trip_data.trip_data
"""

tip_amount = session.execute(tip_amount_query)

tip_amount_df = pd.DataFrame(list(tip_amount))

tip_amount_df = tip_amount_df.groupby("passenger_count").mean().reset_index()



### Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(tip_amount_df["passenger_count"], tip_amount_df["tip_amount"])

ax.set_xlabel("Passenger Count")

ax.set_ylabel("Average Tip Amount")

ax.set_title("Average Tip Amount Per Passenger Count")

plt.show()

## What are the best 5 locations for drivers to pick up passengers from?

In [ ]:
pickup_location_query = """
SELECT
    pickup_location_id,
    COUNT(pickup_location_id) AS count
FROM
    taxi_trip_data.trip_data
GROUP BY
    pickup_location_id
ORDER BY
    count DESC
LIMIT 5
"""

pickup_location = session.execute(pickup_location_query)

pickup_location_df = pd.DataFrame(list(pickup_location))

pickup_location_df

### Closing Connection

In [38]:
# Close the connection
cluster.shutdown()